# HIRA 빅데이터 상담 챗봇 - SOLAR-10.7B LoRA 학습

**프로젝트**: HIRA 빅데이터 상담 챗봇  
**모델**: SOLAR-10.7B-v1.0  
**방법**: LoRA (Low-Rank Adaptation)  
**환경**: A100 80G x2, PyTorch 2.1, JupyterLab

---

## 📋 목차

1. [환경 설정](#1-환경-설정)
2. [데이터 로드 및 전처리](#2-데이터-로드-및-전처리)
3. [모델 및 토크나이저 로드](#3-모델-및-토크나이저-로드)
4. [LoRA 설정](#4-lora-설정)
5. [학습 설정](#5-학습-설정)
6. [학습 실행](#6-학습-실행)
7. [모델 저장 및 평가](#7-모델-저장-및-평가)
8. [추론 테스트](#8-추론-테스트)

## 1. 환경 설정

### 1.1 필요 라이브러리 임포트

In [ ]:
# bitsandbytes 차단 (폐쇄망 환경)
import sys
import os
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
sys.modules['bitsandbytes'] = None

import torch
import json
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from torch.utils.data import Dataset, DataLoader

print("✅ 라이브러리 임포트 완료")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

### 1.2 경로 설정 (유연한 구성)

In [ ]:
# ========================================
# 경로 설정 (환경에 맞게 수정)
# ========================================

# 프로젝트 루트 (현재 노트북 위치 기준)
PROJECT_ROOT = Path().absolute().parent

# 모델 경로 (SOLAR-10.7B 모델이 저장된 위치)
MODEL_PATH = PROJECT_ROOT / "solar_10.7b_package" / "model"
# 또는 절대 경로 사용:
# MODEL_PATH = Path("/path/to/solar_10.7b_package/model")

# 데이터 경로
DATA_PATH = PROJECT_ROOT / "workspace" / "data" / "hira" / "cleaned"

# 출력 경로
OUTPUT_PATH = PROJECT_ROOT / "workspace" / "models" / "solar_hira_lora"
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print("📂 경로 설정:")
print(f"  프로젝트 루트: {PROJECT_ROOT}")
print(f"  모델 경로: {MODEL_PATH}")
print(f"  데이터 경로: {DATA_PATH}")
print(f"  출력 경로: {OUTPUT_PATH}")

# 경로 존재 확인
if not MODEL_PATH.exists():
    print(f"⚠️ 경고: 모델 경로가 존재하지 않습니다: {MODEL_PATH}")
if not DATA_PATH.exists():
    print(f"⚠️ 경고: 데이터 경로가 존재하지 않습니다: {DATA_PATH}")
    print(f"  → 테스트 데이터를 자동 생성합니다.")

### 1.3 GPU 환경 확인

In [ ]:
# GPU 정보 출력
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("🖥️ 하드웨어 정보:")
print(f"  Device: {device}")

if torch.cuda.is_available():
    print(f"  GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"    - VRAM: {props.total_memory / 1024**3:.1f} GB")
        print(f"    - Compute Capability: {props.major}.{props.minor}")
    print(f"  CUDA 버전: {torch.version.cuda}")
else:
    print("  ⚠️ GPU를 사용할 수 없습니다. CPU로 실행됩니다.")

# 메모리 최적화 설정
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

print("\n✅ GPU 최적화 설정 완료")

## 2. 데이터 로드 및 전처리

### 2.1 Dataset 클래스 정의

In [ ]:
class HIRADataset(Dataset):
    """
    HIRA 건강보험 데이터셋
    
    형식: JSONL (JSON Lines)
    구조: {"instruction": ..., "input": ..., "output": ...}
    """
    
    def __init__(self, file_path, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []
        
        # JSONL 파일 로드
        if file_path.exists():
            with open(file_path, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, 1):
                    line = line.strip()
                    if not line:
                        continue
                    try:
                        self.data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"⚠️ 라인 {line_num} JSON 파싱 오류: {e}")
                        continue
        
        print(f"📂 로드 완료: {file_path.name} ({len(self.data)}개 샘플)")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        
        # SOLAR 프롬프트 형식
        instruction = sample.get('instruction', '').strip()
        input_text = sample.get('input', '').strip()
        output_text = sample.get('output', '').strip()
        
        # 프롬프트 구성
        if input_text:
            prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}"
        else:
            prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output_text}"
        
        # 토크나이징
        encoding = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()
        
        # 레이블 생성 (입력과 동일, padding은 -100으로)
        labels = input_ids.clone()
        labels[attention_mask == 0] = -100
        
        # Response 부분만 학습 (선택사항)
        # Response 시작 위치 찾기
        response_token_ids = self.tokenizer.encode("### Response:", add_special_tokens=False)
        response_start_idx = None
        
        search_limit = len(input_ids) - len(response_token_ids) + 1
        if search_limit > 0:
            for i in range(search_limit):
                if input_ids[i:i + len(response_token_ids)].tolist() == response_token_ids:
                    response_start_idx = i + len(response_token_ids)
                    break
        
        # Instruction/Input 부분은 학습하지 않음
        if response_start_idx is not None:
            labels[:response_start_idx] = -100
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

print("✅ HIRADataset 클래스 정의 완료")

### 2.2 샘플 데이터 확인

In [ ]:
# 데이터 파일 경로
train_file = DATA_PATH / "train.jsonl"
val_file = DATA_PATH / "val.jsonl"

# 파일 존재 확인
if train_file.exists():
    # 첫 3개 샘플 확인
    print("📄 데이터 샘플 미리보기:\n")
    with open(train_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 3:
                break
            sample = json.loads(line)
            print(f"[샘플 {i+1}]")
            print(f"Instruction: {sample.get('instruction', '')}")
            print(f"Input: {sample.get('input', '')}")
            print(f"Output: {sample.get('output', '')[:100]}...")
            print()
else:
    print("⚠️ 학습 데이터가 없습니다. 테스트 데이터를 생성합니다.")
    
    # 테스트 데이터 생성
    DATA_PATH.mkdir(parents=True, exist_ok=True)
    
    test_data = [
        {
            "instruction": "건강보험 빅데이터 전문가로서 정확하게 답변하세요.",
            "input": "2023년 MRI 검사 건수는 얼마나 되나요?",
            "output": "2023년 건강보험 적용 MRI 검사는 총 4,251,032건이 시행되었습니다. 이는 전년 대비 약 3.2% 증가한 수치입니다."
        },
        {
            "instruction": "건강보험 제도를 일반인이 이해하기 쉽게 설명하세요.",
            "input": "DRG가 무엇인가요?",
            "output": "DRG(진단명 기준 환자군 분류)는 같은 질병으로 입원한 환자들을 그룹으로 묶어 미리 정해진 금액을 지불하는 제도입니다. 예를 들어 맹장 수술을 받으면 입원 기간이나 사용한 약에 관계없이 일정 금액만 내게 됩니다."
        },
        {
            "instruction": "건강보험 빅데이터 전문가로서 정확하게 답변하세요.",
            "input": "2023년 전체 진료비는?",
            "output": "2023년 건강보험 총 진료비는 약 125조 2,450억원으로 집계되었습니다. 이는 전년 대비 약 8.2% 증가한 수치입니다."
        }
    ] * 100  # 300개 생성
    
    # Train/Val 분할 (90:10)
    split_idx = int(len(test_data) * 0.9)
    
    with open(train_file, 'w', encoding='utf-8') as f:
        for item in test_data[:split_idx]:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    with open(val_file, 'w', encoding='utf-8') as f:
        for item in test_data[split_idx:]:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"✅ 테스트 데이터 생성 완료")
    print(f"  Train: {train_file} ({split_idx}개)")
    print(f"  Val: {val_file} ({len(test_data) - split_idx}개)")

## 3. 모델 및 토크나이저 로드

### 3.1 토크나이저 로드

In [ ]:
print("🔄 토크나이저 로드 중...")

tokenizer = AutoTokenizer.from_pretrained(
    str(MODEL_PATH),
    trust_remote_code=True,
    local_files_only=True
)

# Padding 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"  pad_token을 eos_token으로 설정: {tokenizer.eos_token}")

tokenizer.padding_side = "right"

print("✅ 토크나이저 로드 완료")
print(f"  Vocab 크기: {len(tokenizer)}")
print(f"  Pad token: {tokenizer.pad_token}")
print(f"  EOS token: {tokenizer.eos_token}")

### 3.2 SOLAR 모델 로드

In [ ]:
print("🔄 SOLAR-10.7B 모델 로드 중...")
print(f"  경로: {MODEL_PATH}")

# bfloat16으로 로드 (A100 최적)
model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_PATH),
    torch_dtype=torch.bfloat16,
    device_map="auto",  # 자동으로 GPU에 분산
    trust_remote_code=True,
    local_files_only=True,
    use_cache=False  # 학습 시 필수
)

# Gradient checkpointing (메모리 절약)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

print("✅ 모델 로드 완료")
print(f"  파라미터 수: {model.num_parameters() / 1e9:.2f}B")
print(f"  dtype: {model.dtype}")
print(f"  device: {next(model.parameters()).device}")

## 4. LoRA 설정

### 4.1 LoRA Config 정의

In [ ]:
print("🔧 LoRA 설정 중...")

# LoRA 하이퍼파라미터
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
        "gate_proj", "up_proj", "down_proj"  # MLP
    ],
    bias="none",
    modules_to_save=["embed_tokens", "lm_head"]  # 임베딩도 학습 (선택사항)
)

print("LoRA 설정:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Target Modules: {lora_config.target_modules}")

### 4.2 LoRA 적용

In [ ]:
# LoRA 어댑터 추가
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 정보 출력
model.print_trainable_parameters()

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print(f"\n✅ LoRA 적용 완료")
print(f"  학습 가능 파라미터: {trainable_params:,} ({100 * trainable_params / all_params:.2f}%)")
print(f"  전체 파라미터: {all_params:,}")

## 5. 학습 설정

### 5.1 하이퍼파라미터 정의

In [ ]:
# 학습 설정
config = {
    "batch_size": 2,  # A100 80G에서 안전한 배치 크기
    "gradient_accumulation_steps": 4,  # 실효 배치 = 8
    "learning_rate": 2e-4,  # LoRA 권장 학습률
    "num_epochs": 10,
    "max_length": 512,
    "weight_decay": 0.01,
    "max_grad_norm": 1.0,
    "warmup_ratio": 0.1,
    "save_epochs": 2,  # 2 에폭마다 체크포인트 저장
    "eval_steps": 50,  # 50 스텝마다 검증
    "logging_steps": 10,
    "patience": 5  # Early stopping patience
}

print("⚙️ 학습 설정:")
for key, value in config.items():
    print(f"  {key}: {value}")

### 5.2 데이터셋 및 DataLoader 생성

In [ ]:
print("📂 데이터셋 로드 중...\n")

# Dataset 생성
train_dataset = HIRADataset(train_file, tokenizer, config["max_length"])
val_dataset = HIRADataset(val_file, tokenizer, config["max_length"])

print(f"\n데이터 분포:")
print(f"  학습: {len(train_dataset)}개")
print(f"  검증: {len(val_dataset)}개")

# DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"\n✅ DataLoader 생성 완료")
print(f"  학습 배치 수: {len(train_loader)}")
print(f"  검증 배치 수: {len(val_loader)}")

### 5.3 Optimizer 및 Scheduler 설정

In [ ]:
# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config["learning_rate"],
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=config["weight_decay"]
)

# Learning Rate Scheduler (Cosine Annealing)
from torch.optim.lr_scheduler import CosineAnnealingLR

total_steps = config["num_epochs"] * len(train_loader)
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=total_steps,
    eta_min=config["learning_rate"] * 0.1
)

print("✅ Optimizer 및 Scheduler 설정 완료")
print(f"  총 학습 스텝: {total_steps}")
print(f"  초기 학습률: {config['learning_rate']:.2e}")
print(f"  최소 학습률: {config['learning_rate'] * 0.1:.2e}")

## 6. 학습 실행

### 6.1 평가 함수 정의

In [ ]:
def evaluate(model, val_loader, device):
    """
    검증 데이터셋에서 모델 평가
    """
    model.eval()
    total_loss = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            total_loss += outputs.loss.item()
            num_batches += 1
    
    model.train()
    return total_loss / num_batches if num_batches > 0 else float('inf')

print("✅ 평가 함수 정의 완료")

### 6.2 학습 루프 실행

In [ ]:
print("\n" + "="*80)
print("🚀 학습 시작")
print("="*80 + "\n")

# 학습 히스토리
history = {
    "train_loss": [],
    "val_loss": [],
    "learning_rate": [],
    "epochs": []
}

# Early stopping 변수
best_val_loss = float('inf')
patience_counter = 0
start_time = datetime.now()

model.train()

for epoch in range(config["num_epochs"]):
    print(f"\n{'='*80}")
    print(f"Epoch {epoch + 1}/{config['num_epochs']}")
    print(f"{'='*80}")
    
    epoch_loss = 0
    optimizer.zero_grad()
    steps_since_opt_step = 0
    
    progress_bar = tqdm(train_loader, desc=f"Training")
    
    for step, batch in enumerate(progress_bar):
        # 데이터 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss / config["gradient_accumulation_steps"]
        loss.backward()
        
        epoch_loss += loss.item() * config["gradient_accumulation_steps"]
        steps_since_opt_step += 1
        
        # Gradient accumulation
        if steps_since_opt_step == config["gradient_accumulation_steps"]:
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), config["max_grad_norm"])
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            steps_since_opt_step = 0
            
            # Progress bar 업데이트
            current_lr = scheduler.get_last_lr()[0]
            progress_bar.set_postfix({
                "loss": f"{loss.item() * config['gradient_accumulation_steps']:.4f}",
                "lr": f"{current_lr:.2e}"
            })
    
    # 마지막 남은 gradient 처리
    if steps_since_opt_step > 0:
        torch.nn.utils.clip_grad_norm_(model.parameters(), config["max_grad_norm"])
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    # Epoch 평균 손실
    avg_train_loss = epoch_loss / len(train_loader)
    
    # Validation
    print(f"\n📊 Epoch {epoch + 1} 평가 중...")
    val_loss = evaluate(model, val_loader, device)
    
    # 히스토리 저장
    history["train_loss"].append(avg_train_loss)
    history["val_loss"].append(val_loss)
    history["learning_rate"].append(scheduler.get_last_lr()[0])
    history["epochs"].append(epoch + 1)
    
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {val_loss:.4f}")
    print(f"  LR:         {scheduler.get_last_lr()[0]:.2e}")
    
    # Best model 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        
        checkpoint_path = OUTPUT_PATH / "best_model"
        model.save_pretrained(checkpoint_path)
        tokenizer.save_pretrained(checkpoint_path)
        
        print(f"  ✅ Best model 저장: {checkpoint_path}")
    else:
        patience_counter += 1
        print(f"  ⚠️ Validation loss 개선 없음. Patience: {patience_counter}/{config['patience']}")
        
        # Early stopping
        if patience_counter >= config["patience"]:
            print(f"\n🛑 Early stopping at epoch {epoch + 1}")
            print(f"   Best validation loss: {best_val_loss:.4f}")
            break
    
    # 주기적 체크포인트 저장
    if (epoch + 1) % config["save_epochs"] == 0:
        checkpoint_path = OUTPUT_PATH / f"checkpoint-epoch-{epoch+1}"
        model.save_pretrained(checkpoint_path)
        tokenizer.save_pretrained(checkpoint_path)
        print(f"  💾 Checkpoint 저장: {checkpoint_path}")

# 학습 종료
end_time = datetime.now()
training_time = end_time - start_time

print("\n" + "="*80)
print("✅ 학습 완료!")
print("="*80)
print(f"  Best Validation Loss: {best_val_loss:.4f}")
print(f"  총 소요 시간: {training_time}")
print(f"  모델 저장 위치: {OUTPUT_PATH}")

### 6.3 학습 곡선 시각화

In [ ]:
# 학습 곡선 플롯
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss 곡선
ax1.plot(history["epochs"], history["train_loss"], label="Train Loss", marker='o')
ax1.plot(history["epochs"], history["val_loss"], label="Val Loss", marker='s')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("Training and Validation Loss")
ax1.legend()
ax1.grid(True)

# Learning Rate 곡선
ax2.plot(history["epochs"], history["learning_rate"], marker='o', color='green')
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Learning Rate")
ax2.set_title("Learning Rate Schedule")
ax2.set_yscale('log')
ax2.grid(True)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / "training_curves.png", dpi=150)
plt.show()

print(f"✅ 학습 곡선 저장: {OUTPUT_PATH / 'training_curves.png'}")

## 7. 모델 저장 및 평가

### 7.1 최종 모델 저장

In [ ]:
# 최종 모델 저장
final_path = OUTPUT_PATH / "final_model"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)

print(f"✅ 최종 모델 저장: {final_path}")

# 학습 히스토리 저장
history_file = OUTPUT_PATH / "training_history.json"
with open(history_file, 'w', encoding='utf-8') as f:
    json.dump(history, f, indent=2, ensure_ascii=False)

print(f"✅ 학습 히스토리 저장: {history_file}")

# 설정 저장
config_file = OUTPUT_PATH / "training_config.json"
with open(config_file, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"✅ 학습 설정 저장: {config_file}")

### 7.2 학습 결과 요약

In [ ]:
print("\n" + "="*80)
print("📊 학습 결과 요약")
print("="*80)

print(f"\n모델 정보:")
print(f"  베이스 모델: SOLAR-10.7B-v1.0")
print(f"  LoRA Rank: {lora_config.r}")
print(f"  학습 가능 파라미터: {trainable_params:,}")

print(f"\n학습 데이터:")
print(f"  Train: {len(train_dataset)}개")
print(f"  Val: {len(val_dataset)}개")

print(f"\n학습 설정:")
print(f"  Epochs: {len(history['epochs'])}")
print(f"  Batch Size: {config['batch_size']}")
print(f"  Gradient Accumulation: {config['gradient_accumulation_steps']}")
print(f"  Learning Rate: {config['learning_rate']:.2e}")

print(f"\n최종 성능:")
print(f"  Best Val Loss: {best_val_loss:.4f}")
print(f"  Final Train Loss: {history['train_loss'][-1]:.4f}")
print(f"  Final Val Loss: {history['val_loss'][-1]:.4f}")

print(f"\n저장 위치:")
print(f"  Best Model: {OUTPUT_PATH / 'best_model'}")
print(f"  Final Model: {OUTPUT_PATH / 'final_model'}")
print(f"  History: {history_file}")

print("\n" + "="*80)

## 8. 추론 테스트

### 8.1 Best 모델 로드

In [ ]:
print("🔄 Best 모델 로드 중...")

# 베이스 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_PATH),
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True
)

# LoRA 어댑터 로드
best_model_path = OUTPUT_PATH / "best_model"
test_model = PeftModel.from_pretrained(base_model, str(best_model_path))
test_model.eval()

print(f"✅ 모델 로드 완료: {best_model_path}")

### 8.2 추론 함수 정의

In [ ]:
def generate_response(question, max_new_tokens=256, temperature=0.3):
    """
    질문에 대한 답변 생성
    """
    # 프롬프트 구성
    prompt = f"### Instruction:\n건강보험 빅데이터 전문가로서 정확하게 답변하세요.\n\n### Input:\n{question}\n\n### Response:\n"
    
    # 토크나이징
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)
    
    # 생성
    with torch.no_grad():
        outputs = test_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.85,
            top_k=40,
            repetition_penalty=1.15,
            no_repeat_ngram_size=3,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Response 부분만 추출
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    
    return response

print("✅ 추론 함수 정의 완료")

### 8.3 테스트 질문으로 평가

In [ ]:
# 테스트 질문 리스트
test_questions = [
    "2023년 MRI 검사 건수는 얼마나 되나요?",
    "DRG가 무엇인가요?",
    "고혈압 환자 수는?",
    "건강보험 총 진료비는 얼마인가요?",
    "요양병원과 종합병원의 차이는?"
]

print("\n" + "="*80)
print("🧪 모델 테스트")
print("="*80 + "\n")

for i, question in enumerate(test_questions, 1):
    print(f"[질문 {i}] {question}")
    response = generate_response(question)
    print(f"[답변] {response}")
    print("\n" + "-"*80 + "\n")

### 8.4 인터랙티브 테스트

In [ ]:
# 직접 질문해보기 (이 셀을 여러 번 실행 가능)
print("💬 질문을 입력하세요 (종료: 'quit')\n")

while True:
    question = input("Q: ").strip()
    
    if question.lower() in ['quit', 'exit', '종료']:
        print("\n테스트를 종료합니다.")
        break
    
    if not question:
        continue
    
    response = generate_response(question)
    print(f"\nA: {response}\n")
    print("-" * 80 + "\n")

## 완료!

HIRA 빅데이터 상담 챗봇 모델 학습이 완료되었습니다.

### 다음 단계

1. **모델 평가**: `HIRA_Evaluation.ipynb` 노트북 실행
2. **인터페이스 배포**: `HIRA_Interface.ipynb` 노트북 실행
3. **모델 개선**: 데이터 증강 및 하이퍼파라미터 튜닝

### 저장된 파일

- Best Model: `workspace/models/solar_hira_lora/best_model/`
- Training History: `workspace/models/solar_hira_lora/training_history.json`
- Training Curves: `workspace/models/solar_hira_lora/training_curves.png`